# Convert GEOJSON to XLSX and vice versa

In [3]:
import os  # To work with file paths
import json
import pandas as pd

def geojson_to_xlsx(geojson_file):
    # Load GeoJSON data
    with open(geojson_file, 'r') as f:
        geojson_data = json.load(f)
    
    # Extract features
    features = geojson_data['features']
    
    # Create a DataFrame from the features
    df = pd.json_normalize(features)

    xlsx_file = os.path.splitext(geojson_file)[0] + ".xlsx"

    # Save DataFrame to Excel file
    df.to_excel(xlsx_file, index=False)
    print(f"GeoJSON data has been converted to {xlsx_file}")

In [5]:
import os  # To work with file paths
import json
import pandas as pd

def xlsx_to_geojson_beautifued(xlsx_file):
    # Extract the name from the xlsx_file
    name = os.path.splitext(os.path.basename(xlsx_file))[0]
    geojson_file = os.path.splitext(xlsx_file)[0] + ".geojson"
    
    # Load Excel data into a DataFrame
    df = pd.read_excel(xlsx_file)

    # Ensure required columns are present
    required_columns = ['geometry.type', 'geometry.coordinates']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Missing required column: '{col}' in the Excel file.")

    # Convert each row to a GeoJSON feature
    features = []
    for _, row in df.iterrows():
        geometry = {
            "type": row['geometry.type'],
            "coordinates": json.loads(row['geometry.coordinates'])  # Parse coordinates from string to Python object
        }
        # Collect all properties except the geometry columns
        property_columns = [col for col in df.columns if col not in required_columns]
        properties = row[property_columns].to_dict()

        # replacing the text "properties." from each column name
        properties = {col.replace('properties.',''):value for col,value in properties.items()}

        # Construct GeoJSON feature
        feature = {
            "type": "Feature",
            "properties": properties,
            "geometry": geometry
        }
        features.append(feature)

    # Create the GeoJSON structure
    geojson_data = {
        "type": "FeatureCollection",
        "name": name,
        "crs": {
            "type": "name",
            "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" }
        },
        "features": features
    }

    # Save GeoJSON data to file
    with open(geojson_file, 'w') as f:
        json.dump(geojson_data, f, indent=4)
    print(f"Excel data has been converted to {geojson_file}")


In [7]:
path_geojson = r'C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\Plots Corners.geojson'
path_xlsx = r'C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\Plots Corners.xlsx'

# path = r"D:\OLF PHENO\GCPs Pheno 2019-2023.geojson"

In [9]:
geojson_to_xlsx(path_geojson)


GeoJSON data has been converted to C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\Plots Corners.xlsx


In [ ]:
xlsx_to_geojson_beautifued(path_xlsx)

# MultiPolygon GEOJSON to CSV for converting sowing plots to MyMaps upload files

## Improving

In [114]:
import json
from shapely.geometry import shape, Polygon, MultiPolygon

def round_coords(coords, decimals):
    return [round(coord, decimals) for coord in coords]

def round_geometry(geom, decimals):
    if isinstance(geom, Polygon):
        exterior = [round_coords(pt, decimals) for pt in list(geom.exterior.coords)]
        interiors = [[round_coords(pt, decimals) for pt in ring.coords] for ring in geom.interiors]
        return Polygon(exterior, interiors)
    elif isinstance(geom, MultiPolygon):
        return MultiPolygon([round_geometry(p, decimals) for p in geom.geoms])
    else:
        raise TypeError("Unsupported geometry type.")

def geojson_to_csv_manual(geojson_file, csv_file, decimal_places=5):
    with open(geojson_file, 'r') as f:
        data = json.load(f)

    features = data['features']
    rows = []
    headers_written = False

    with open(csv_file, 'w', encoding='utf-8') as out:
        for feature in features:
            geom = shape(feature['geometry'])
            geom = round_geometry(geom, decimal_places)

            # Flatten multipolygon by taking first polygon
            if geom.geom_type == "MultiPolygon":
                geom = list(geom.geoms)[0]

            wkt_str = f'"{geom.wkt}"'  # ONLY ONE PAIR OF QUOTES

            props = feature.get('properties', {})

            # Write headers once
            if not headers_written:
                headers = ['WKT'] + list(props.keys())
                out.write(','.join(headers) + '\n')
                headers_written = True

            # Write values
            values = [wkt_str] + [str(props[k]) if props[k] is not None else '' for k in props]
            out.write(','.join(values) + '\n')

    print(f"GeoJSON manually written to CSV without extra quotes: {csv_file}")


In [116]:
path_geojson = r'C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\25 Smartwheat drought.geojson'
path_csv = r'C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\25 Smartwheat drought.csv'


In [ ]:
path.join

In [76]:
geojson_to_csv_manual(path_geojson, path_csv, decimal_places=8)


GeoJSON manually written to CSV without extra quotes: C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\25 Smartwheat drought.CSV


## batch_geojson_to_csv

In [118]:
import os

def batch_geojson_to_csv(folder_path, decimal_places=8):
    import os
    import glob

    # Ensure the output subfolder exists
    output_folder = os.path.join(folder_path, 'CSV for MyMaps')
    os.makedirs(output_folder, exist_ok=True)

    # Get all .geojson files in the folder
    geojson_files = glob.glob(os.path.join(folder_path, '*.geojson'))

    if not geojson_files:
        print("No GeoJSON files found in folder.")
        return

    for geojson_file in geojson_files:
        filename = os.path.splitext(os.path.basename(geojson_file))[0]
        csv_file = os.path.join(output_folder, f"{filename}.csv")
        print(f"Processing: {geojson_file} → {csv_file}")
        geojson_to_csv_manual(geojson_file, csv_file, decimal_places)

    print(f"Finished converting {len(geojson_files)} files.")


In [120]:
path_geojson_all = r"C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\GeoJSON"

In [122]:
batch_geojson_to_csv(path_geojson_all, decimal_places=8)

Processing: C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\GeoJSON\25 Aakerbonner erter - GN.geojson → C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\GeoJSON\CSV for MyMaps\25 Aakerbonner erter - GN.csv
GeoJSON manually written to CSV without extra quotes: C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\GeoJSON\CSV for MyMaps\25 Aakerbonner erter - GN.csv
Processing: C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\GeoJSON\25 Delgjodsling NIBIO Bygg.geojson → C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\GeoJSON\CSV for MyMaps\25 Delgjodsling NIBIO Bygg.csv
Geo

TypeError: Unsupported geometry type.

## Adding name of field to the csv files

In [124]:
import os
import csv

def update_csv_name_column(folder_path):
    # Get all .csv files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            csv_path = os.path.join(folder_path, filename)
            base_name = os.path.splitext(filename)[0]

            updated_rows = []
            with open(csv_path, 'r', encoding='utf-8') as f:
                reader = csv.DictReader(f)
                fieldnames = reader.fieldnames

                # Ensure NAME column exists
                if 'NAME' not in fieldnames:
                    fieldnames.append('NAME')

                for row in reader:
                    row['NAME'] = base_name
                    updated_rows.append(row)

            # Write updated rows back to the same file
            with open(csv_path, 'w', encoding='utf-8', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(updated_rows)

            print(f"Updated NAME column in: {filename}")


In [126]:
path_csv_ = r"C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\GeoJSON\CSV for MyMaps"

In [128]:
update_csv_name_column(path_csv_)


Updated NAME column in: 25 Aakerbonner erter - GN.csv
Updated NAME column in: 25 Delgjodsling NIBIO Bygg.csv
Updated NAME column in: 25 Diversityoats.csv
Updated NAME column in: 25 Gene2Bread Hvete Wheat.csv
Updated NAME column in: 25 OatsFrontier.csv
Updated NAME column in: 25 Oppformering gene2bread.csv
Updated NAME column in: 25 PhenoCrop Avlingsregistrering.csv
Updated NAME column in: 25 ProteinBar 20 sorter 7m.csv
Updated NAME column in: 25 Smartwheat drought.csv
Updated NAME column in: 25 Soraas- ProteinBar.csv
Updated NAME column in: 25 SproutResist m-vanning 1-2.csv
Updated NAME column in: 25 Sproutresist U-vanning.csv
Updated NAME column in: 25 Verresistens.csv
Updated NAME column in: 25A01A04 Havre Oats.csv
Updated NAME column in: 25G01A04 Bygg Barley.csv


TypeError: argument of type 'NoneType' is not iterable

## Merge all CSVs in a folder into a single CSV

In [167]:
import os
import pandas as pd

def merge_and_split_csv(input_folder, output_folder, max_rows_per_file=2000):
    # Create the output subfolder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # List to collect all DataFrames
    dfs = []
    total_rows = 0

    # Loop through all CSV files in the folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.csv'):
            file_path = os.path.join(input_folder, filename)
            # Load CSV into a DataFrame
            df = pd.read_csv(file_path)
            dfs.append(df)
            total_rows += len(df)

    # Concatenate all DataFrames into one
    merged_df = pd.concat(dfs, ignore_index=True)

    # Split merged DataFrame into chunks of max_rows_per_file
    current_rows = 0
    part_number = 1
    current_df = pd.DataFrame(columns=merged_df.columns)  # Start with empty DataFrame for each part

    for df in dfs:
        current_rows += len(df)
        
        # Check if adding this DataFrame exceeds max rows per file
        if current_rows <= max_rows_per_file:
            # Append to current part
            current_df = pd.concat([current_df, df], ignore_index=True)
        else:
            # Save the current part to file and reset for next part
            output_file = os.path.join(output_folder, f"merged_part_{part_number}.csv")
            current_df.to_csv(output_file, index=False)
            print(f"✅ Split file saved: {output_file}")
            
            # Start a new part with the current DataFrame
            part_number += 1
            current_df = df
            current_rows = len(df)

    # Save the last part if it has any rows
    if len(current_df) > 0:
        output_file = os.path.join(output_folder, f"merged_part_{part_number}.csv")
        current_df.to_csv(output_file, index=False)
        print(f"✅ Split file saved: {output_file}")

    print(f"All CSV files merged and split into {part_number} files.")



In [169]:
merge_and_split_csv(path_csv_, os.path.join(path_csv_,"merged_output"))


✅ Split file saved: C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\GeoJSON\CSV for MyMaps\merged_output\merged_part_1.csv
✅ Split file saved: C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\GeoJSON\CSV for MyMaps\merged_output\merged_part_2.csv
✅ Split file saved: C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\GeoJSON\CSV for MyMaps\merged_output\merged_part_3.csv
✅ Split file saved: C:\Users\muij\OneDrive - Norwegian University of Life Sciences\Documents\+PROJECTS\1. OPERATIONS MANAGEMENT\2025 - UAV\2. Sowing Maps\GeoJSON\CSV for MyMaps\merged_output\merged_part_4.csv
All CSV files merged and split into 4 files.


C:\NMBU\TEMP\ipykernel_61776\2574062726.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_df = pd.concat([current_df, df], ignore_index=True)
